In [1]:
import os
from rqalpha_plus import run_func
from rqalpha_plus.api import *


config = {
    'extra': {
        'log_level': 'error',
    },
    'base': {
        'matching_type': 'current_bar',
        'start_date': '2014-01-01',
        'end_date': '2018-04-01',
        'benchmark': '000001.XSHE',
        'frequency': '1m',
        'accounts': {
                 'future': 1000000
      
             }
    },
    'mod': {
        'sys_progress': {
            'enabled': True,
            'show': True,
        },
        'sys_analyser': {
            'enabled': True,
        },
    },
}

tasks = []
for _period_a in range(20, 100, 10):
    for ratio in range(1, 2, 1):
        tasks.append((config, _period_a, ratio))


def run_bt(config, _period_a, ratio):
    print(_period_a,ratio)
      # 可以自己import我们平台支持的第三方python模块，比如pandas、numpy等。
    from sklearn.linear_model import LinearRegression
    import numpy as np
    import pandas as pd
    # 在这个方法中编写任何的初始化逻辑。context对象将会在你的算法策略的任何方法之间做传递。
    def init(context):
        # context内引入全局变量s1
        # 螺纹钢，铁矿石,5
        pairs1 = ["I","RB"]
        # 焦煤，焦炭,1
        pairs2 = ["JM","J"]
        # 鸡蛋，玉米，豆粕
        pairs3 = ["JD","M"]
        # 银，金,15
        pairs5 = ["AU","AG"]
        # 铜，锌,2
        pairs6 = ["ZN","CU"]

        context.pairs = pairs2


        context.observed_periods1 = 600

        # context.over_up_point = False
        # context.under_low_point = False
        context.observed_periods_a = _period_a

        context.bar_count = 0

        context.entry_multiply = 2.5
        context.command = ["N","D"]

        # 订阅连续主力
        subscribe([i +"88" for i in context.pairs])
        # 实时打印日志

    # before_trading此函数会在每天策略交易开始前被调用，当天只会被调用一次
    def before_trading(context):
        context.bar_count = 0
        # 计算对冲比例
        # context.ratio
        # calc_pairs_ratio(context)
        context.ratio = ratio
        context.changeSymbol = "N"

        # 日内不会更换主力，因此在交易前判断是否更换主力
        # 最新主力保留在 context.latest_futures


    def check(context,bar_dict):

        item1,item2 = context.latest_futures


        # calculate the spread of items
        _price1 = history_bars(item1,context.observed_periods_a,'1m','close')
        _price2 = history_bars(item2,context.observed_periods_a,'1m','close')

        _spread = _price2 - context.ratio * _price1
        _spread_std = _spread.std()
        _spread_mean = _spread.mean()

        up_point = _spread_mean + context.entry_multiply * _spread_std
        low_point = _spread_mean - context.entry_multiply * _spread_std

        plot("up_point",up_point)
        plot("low_point",low_point)



        cur_price_1 = bar_dict[item1].close
        cur_price_2 = bar_dict[item2].close
        _cur_spread = cur_price_2 - context.ratio * cur_price_1
        plot("cur_spread",_cur_spread)

        item1_bqt,item1_sqt = get_positions(context,item1)
        item2_bqt,item2_sqt = get_positions(context,item2)

        if item1_bqt == item1_sqt == item2_bqt == item2_sqt == 0:
            # 当前空仓，需要买入差价 或者卖出价差
            # 价差低于下限，建仓
            # item2 价低，item1价高
            if _cur_spread<=low_point:
#                 print("买入价差")
                context.command = ["spread_B","N"]
            elif _cur_spread>=up_point:
#                 print("卖出价差")
                context.command = ["spread_S","N"]


        # 之前已经买入价差，判断当前是否需要平仓
        elif item2_bqt >0 and item1_sqt >0 and context.command[0] == "spread_B":
            if _cur_spread >= _spread_mean:
#                 print("平多仓")
                context.command = ["close","N"]

        # 之前已经卖出价差，判断当前是否需要平仓
        elif item1_bqt >0 and item2_sqt >0 and context.command[0] == "spread_S":
            if _cur_spread <= _spread_mean:
#                 print("平空仓")
                context.command = ["close","N"]

        # else:
        #     context.command = ["N","D"]

    def contracts_order(context,bar_dict):

        item1,item2 = context.latest_futures

        item1_bqt,item1_sqt = get_positions(context,item1)
        item2_bqt,item2_sqt = get_positions(context,item2)

        if context.command[0] == "spread_B" and context.command[1] == "N":
            bqt_item2_left = 1-item2_bqt
            sqt_item1_left = context.ratio - item1_sqt

            if bqt_item2_left>0:
                buy_open(item2,bqt_item2_left)
            if sqt_item1_left>0:
                sell_open(item1,sqt_item1_left)
            if bqt_item2_left == sqt_item1_left == 0:
                context.command =  ["spread_B","D"]

        if context.command[0] == "spread_S" and context.command[1] == "N":
            bqt_item1_left = 1-item1_bqt
            sqt_item2_left = context.ratio - item2_sqt

            if bqt_item1_left>0:
                buy_open(item1,bqt_item1_left)
            if sqt_item2_left>0:
                sell_open(item2,sqt_item2_left)
            if bqt_item1_left == sqt_item2_left == 0:
                context.command =  ["spread_S","D"]

        if context.command[0] == "close" and context.command[1] == "N":
            feedBack = close_Contracts(context,[item1,item2])          
            if feedBack == "D":
                context.command = ["close","D"]


    # 你选择的期货数据更新将会触发此段逻辑，例如日线或分钟线更新
    def handle_bar(context, bar_dict):
        context.bar_count+=1

        if context.changeSymbol == "N":
            change_symbol(context)

        if context.bar_count>context.observed_periods_a and context.changeSymbol ==  "D" and context.command[1] == "D":
            # 之前的 交易 完成 才进行下一次 交易判断
            check(context,bar_dict)
        if context.command[1] == "N":
            contracts_order(context,bar_dict)

    # after_trading函数会在每天交易结束后被调用，当天只会被调用一次
    def after_trading(context):
        pass


    def calc_pairs_ratio(context):
        pairsx = [i+"88" for i in context.pairs]
        item1_x,item2_x = pairsx
        _price1 = (history_bars(item1_x,context.observed_periods1,'1d','close'))
        _price2 = (history_bars(item2_x,context.observed_periods1,'1d','close'))

        _lenx = min(len(_price1),len(_price2))
        _price1 = _price1[-_lenx:]
        _price2 = _price2[-_lenx:]


        linreg = LinearRegression(fit_intercept = False)
        context.ratio = linreg.fit(_price1.reshape(-1,1),_price2).coef_[0]
        context.ratio = int(context.ratio)

    def close_Contracts(context,contracts):
        feedBacks = list(set(map(lambda x :close_oneContract(context,x),contracts)))
        if len(feedBacks)==1 and feedBacks[0]=="D":
            return "D"


    def change_symbol(context):
        # 当前持有的期货代码
        cur_positions = context.portfolio.positions

        # 最新的主力期货代码
        latest_futures = [get_dominant_future(code,context.now).iloc[-1] for code in context.pairs]
        context.latest_futures = latest_futures
        # 订阅行情
        subscribe(context.latest_futures)
#         print(context.latest_futures)

        # 需要平仓的
        context.to_close = sorted(set(cur_positions)-set(latest_futures))

        if len(context.to_close) == 0:
            context.changeSymbol = "D"
            # logger.info("没有需要强制平仓的期货")
            pass
        else:
            context.changeSymbol = "N"
#             print(context.now,"需要更换合约")
            feedBack = close_Contracts(context,context.to_close)
            if feedBack == "D":
#                 print("换仓已完成")
                context.changeSymbol = "D"
                context.to_close = []
            else:
                pass
#                 print("换仓未完成")


    def close_oneContract(context,s):
        _bqt ,_sqt = get_positions(context,s)

        if _bqt>0:
            sell_close(s,_bqt)
        elif _sqt>0:
            buy_close(s,_sqt)

        _bqt ,_sqt = get_positions(context,s)
        if _bqt == _sqt == 0:
            return "D"

    def get_positions(context,contract):
        positions = context.portfolio.positions[contract]
        return positions.buy_quantity,positions.sell_quantity




    def stoploss(context,bar_dict):
        holding_contracts = sorted(list(context.portfolio.positions.keys()))

        buy_avg_holding_prices = list(map(lambda x: context.portfolio.positions[x].buy_avg_holding_price,holding_contracts))
        sell_avg_holding_prices = list(map(lambda x: context.portfolio.positions[x].sell_avg_holding_price,holding_contracts))

        cur_prices = list(map(lambda x: bar_dict[x].last,holding_contracts))

        for i, contract in enumerate(holding_contracts):
            if buy_avg_holding_prices[i] > cur_prices[i]*1.1:
                clear_long(context,contract)
            if sell_avg_holding_prices[i] < cur_prices[i]*1.1:
                clear_short(context,contract)



    
    
    

            
    name = '{}-{}'.format(_period_a, ratio)
    result = run_func(config=config, init=init, handle_bar=handle_bar,before_trading=before_trading)
    return result, name


results = []
for task in tasks:
    result, name = run_bt(*task)
    results.append((result['sys_analyser'], name))

    
# 从回测结果中提取分析信息
data = []
for result, name in results:
    summary = result['summary']
    data.append({
        'name': name,
        'annualized_returns': summary['annualized_returns'],
        'sharpe': summary['sharpe'],
        'max_drawdown': summary['max_drawdown'],
    })    
    
    
# 分析回测
results_df = pd.DataFrame(data)
 
print('')
print('-' * 50)
print('Sort by sharpe')
print(results_df.sort_values('sharpe', ascending=False)[:10])
 
print('-' * 50)
print('Sort by annualized_returns')
print(results_df.sort_values('annualized_returns', ascending=False)[:10])

from rqdatac import *

20 1

30 1

40 1

50 1

60 1

70 1

80 1

90 1


--------------------------------------------------
Sort by sharpe
   annualized_returns  max_drawdown  name  sharpe
0               0.123         0.005  20-1   5.132
1               0.114         0.010  30-1   3.877
2               0.102         0.008  40-1   2.894
3               0.091         0.033  50-1   1.648
4               0.078         0.024  60-1   1.365
5               0.074         0.035  70-1   1.011
6               0.062         0.037  80-1   0.563
7               0.059         0.038  90-1   0.436
--------------------------------------------------
Sort by annualized_returns
   annualized_returns  max_drawdown  name  sharpe
0               0.123         0.005  20-1   5.132
1               0.114         0.010  30-1   3.877
2               0.102         0.008  40-1   2.894
3               0.091         0.033  50-1   1.648
4               0.078         0.024  60-1   1.365
5               0.074         0.035  70-1   1.011
6      

In [5]:
summaries = pd.DataFrame()
for result, name in results:
  summaries[name] = pd.Series(result["summary"])
summaries.to_pickle("JM-J_summaries.pkl")

In [37]:
trades = {}
for result, name in results:
  trades[name] = (result["trades"].reset_index())
pd.Panel(trades).to_pickle("JM-J-trades.pkl")

In [38]:
future_positions = {}
for result, name in results:
  future_positions[name] = (result["future_positions"].reset_index())
pd.Panel(future_positions).to_pickle("JM-J-future_positions.pkl")

In [40]:
# result["portfolio"]
portfolio = {}
for result, name in results:
  portfolio[name] = (result["portfolio"].reset_index())
pd.Panel(portfolio).to_pickle("JM-J-portfolio.pkl")

In [42]:
# result["portfolio"]
future_account = {}
for result, name in results:
  future_account[name] = (result["future_account"].reset_index())
pd.Panel(future_account).to_pickle("JM-J-future_account.pkl")

In [36]:
result.keys()

dict_keys(['plots', 'future_positions', 'summary', 'trades', 'future_account', 'portfolio', 'benchmark_portfolio'])